In [1]:
import os

os.chdir('..')

In [2]:
from tensorflow import keras

2022-04-01 07:51:08.166657: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-01 07:51:08.166752: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
import tensorflow as tf
from keras import backend as K
from reinforce import Reinforcement
from keras.models import Sequential
from model import Model 
from prediction import Predictor
from predictSMILES import *
from utils import *
import numpy as np

2022-04-01 07:51:42.401728: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-01 07:51:42.455121: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-04-01 07:51:42.455190: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-01 07:51:42.455245: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (seriema): /proc/driver/nvidia/version does not exist


In [4]:
config_file = '/home/joao/documentos/DiverseDRL/configReinforce.json' # Configuration file
property_identifier = 'a2d' # It can be 'a2d', kor', 'qed', 'sas', 'logP', or 'jak2'

os.environ["CUDA_VISIBLE_DEVICES"]="0"
#sets environment variables in the notebook
session = tf.compat.v1.Session()
#encapsulates the environment in which Operation objects
#are executed, and Tensor objects are evaluated
K.set_session(session)
#sets the session on

#Por que precisa inicializar uma sessão?

In [5]:
# load configuration file
configReinforce,exp_time=load_config('/home/joao/documentos/DiverseDRL/configReinforce.json')

main.py -function- load_config


In [6]:
# Load generator object
generator_model = Sequential() #creates a model of plain stack of layers
#where each layer has exactly one input tensor and one output tensor.
generator_model = Model(configReinforce)
#groups layers into an object with training and inference features.
generator_model.model.load_weights("/home/joao/documentos/DiverseDRL/generator_model/LSTM_2layer_adam_d3.hdf5")

FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = '/home/joao/documentos/DiverseDRL/generator_model\LSTM_2layer_adam_d3.hdf5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [48]:
configReinforce.keys()

dict_keys(['exp_name', 'file_path_logP', 'file_path_jak2', 'file_path_a2d', 'file_path_kor', 'file_path_generated', 'output', 'paddSize', 'data_length', 'units', 'num_epochs', 'optimizer', 'seed', 'batch_size', 'validation_split', 'verbose_training', 'dropout', 'learning_rate', 'finetune_sample_num', 'model_name_unbiased', 'model_name_biased', 'n_iterations', 'n_to_generate', 'n_policy', 'threshold_set', 'n_to_draw', 'evaluateSmiles'])

In [9]:
# Initialize lists to evaluate the model
difs = [] # List with the differences between the averages
#of the desired property distributions (G_0 and G_optimized)
divs = [] # List with the internal diversities of the G_optimized generated molecules 
perc_valid = [] # List with the % of valid SMILES generated by G_optimized
    
# To compute SA score or qed it's not necessary to have a Predictor model
if property_identifier != 'sas' and property_identifier != 'qed':
    # Load the Predictor object
    predictor = Predictor(configReinforce,property_identifier)
else:
    predictor = None
#J - what is sas and qed?

Models 0 loaded from disk!
Models 1 loaded from disk!
Models 2 loaded from disk!
Models 3 loaded from disk!
Models 4 loaded from disk!


In [10]:
for k,v in configReinforce.items():
    print(f'{k}:{v}')

exp_name:Model
file_path_logP:/home/joao/documentos/DiverseDRL/data/logP_labels.csv
file_path_jak2:/home/joao/documentos/DiverseDRL/data/jak2_data.csv
file_path_a2d:/home/joao/documentos/DiverseDRL/data/data_clean_a2d.csv
file_path_kor:/home/joao/documentos/DiverseDRL/data/data_clean_kop.csv
file_path_generated:/home/joao/documentos/DiverseDRL/data/generated
output:createdSMILES/newSmiles_
paddSize:65
data_length:0
units:256
num_epochs:16
optimizer:adam
seed:56
batch_size:10
validation_split:0.1
verbose_training:True
dropout:0.3
learning_rate:0.001
finetune_sample_num:10
model_name_unbiased:/home/joao/documentos/DiverseDRL/generator_model\LSTM_2layer_adam_d3.hdf5
model_name_biased:/home/joao/documentos/DiverseDRL/generator_model\LSTM_2layer_adam_d3_updated
n_iterations:75
n_to_generate:5000
n_policy:7
threshold_set:2
n_to_draw:2
evaluateSmiles:True


In [11]:
configReinforce['model_name_unbiased'] = 'generator_model/LSTM_2layer_adam_d3.hdf5'
configReinforce['model_name_biased'] = 'generator_model/LSTM_2layer_adam_d3_updated'
# Create Reinforcement Learning object
RL_obj = Reinforcement(generator_model, predictor,configReinforce,property_identifier)

FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = '/home/joao/documentos/DiverseDRL/generator_model\LSTM_2layer_adam_d3.hdf5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [13]:
    # SMILES generation with unbiased Generator 
#    smiles_original, prediction_original,valid,unique,div = RL_obj.test_generator(configReinforce.n_to_generate,0,True)
    
#      Training Generator with RL    
#    RL_obj.policy_gradient()
    
# SMILES generation after 85 training iterations 
smiles_iteration85,prediction_iteration85,valid,unique,div,perc_desirable = RL_obj.test_generator(configReinforce.n_to_generate,85, False)

....................................
updated model load_weights is DONE!


Generating molecules...:  34%|███▎      | 1676/5000 [22:52<45:22,  1.22it/s]


KeyboardInterrupt: 

In [ ]:
# To directly compare the original and biased models several times, evaluating
# prediction differences, diversity, and validity
for k in range(20):
    print("BIASED GENERATION: " + str(k))
    dif,div,valid,perc_uniq,perc_desirable = RL_obj.compare_models(configReinforce.n_to_generate,True)
    difs.append(dif)
    divs.append(div)
    perc_valid.append(valid)
print("Mean value difference: " + str(np.mean(difs)))
print("Mean value diversity: " + str(np.mean(divs)))
print("Mean value validity: " + str(np.mean(perc_valid)))